In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (16,9)

In [3]:
train = pd.read_csv('../data/gresearch/raw/train.csv')
train['timestamp'] = pd.to_datetime(train.timestamp, unit='s')
asset_info = pd.read_csv('../data/gresearch/raw/asset_details.csv')

In [4]:
asset_info.sort_values('Asset_ID')

,Asset_ID,Weight,Asset_Name
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
0,2,2.397895,Bitcoin Cash
10,3,4.406719,Cardano
13,4,3.555348,Dogecoin
3,5,1.386294,EOS.IO
5,6,5.894403,Ethereum
4,7,2.079442,Ethereum Classic
11,8,1.098612,IOTA
6,9,2.397895,Litecoin


In [5]:
train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,2018-01-01 00:01:00,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,2018-01-01 00:01:00,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,2018-01-01 00:01:00,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,2018-01-01 00:01:00,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,2018-01-01 00:01:00,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [6]:
train.timestamp.min()

Timestamp('2018-01-01 00:01:00')

In [7]:
train.timestamp.max()

Timestamp('2021-09-21 00:00:00')

In [8]:
min_date = '2021-01-21'

In [ ]:
for asset in asset_info.Asset_ID.unique():
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(25,8))
    
    coin = train[train.Asset_ID==asset].loc[min_date:]
    coin = coin.set_index('timestamp')
    #coin_daily = coin.resample('').last()[['Close']]
    #coin_daily['ret'] = coin_daily.Close.diff()/coin_daily.Close.shift(1)
    sns.lineplot(x=coin.index, y=coin.Close, ax=ax[0])
    ax[1].bar(x=coin.index, height=coin.Target, color = ['red' if ret<0 else 'blue' for ret in coin.Target.values])
    fig.suptitle(asset_info[asset_info.Asset_ID==asset].Asset_Name.values[0])

## Scoring

Data splitting pipeline
- take one month
- train (inclusive lagged features)
- predict on the following month?

In [ ]:
def split_monthly():
    """
    returns train_df, test_df
    test df only includes testing data
    """
    
    pass